# Gravity Modelling Exercise for GEOL 5826 (Part B)

Apply what you learned in Part A to model the gravity data from the Winnipeg River survey.

## Environment setup

Run the following code blocks to import the necessary libraries. This may take some time. 


In [ ]:
d_obs = gravity_data=npzread("WR_gravity_data.npy");

In [ ]:
using Pkg
Pkg.status()
Pkg.add("PyPlot")
Pkg.add("NPZ") # Library for loading Numpy files

# Import Libraries
using PyPlot # Plotting
using Base.Iterators # Array operators
using Random;
using NPZ;

# Import User Generated Libraries
include("kernel.jl"); # Forward model library
include("conjugateGradient.jl"); # C.G. solver

## End
Created by Christopher Mancuso. 2024.
Rev 1.05